In [1]:
import sys
sys.path.append('../../preprocess_assets/')

In [2]:
# Main libraries 
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier 
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt

# Our files
from data_shuffling_split import *
from features_extraction import *
from ara_vec_preprocess_configs import *
from ml_modeling import *

In [ ]:
strat_train_set = read_file("train/strat_train_set.csv")
strat_train_set.head()

In [ ]:
x_train_text, x_val_text, y_train, y_val = prepare_data(strat_train_set)

In [ ]:
x_train_text_tokenized = tokenize_using_nltk_TreebankWordTokenizer(x_train_text)

print("Before Tokenization : \n", x_train_text[:3])
print("="*50)
print("After Tokenization : \n", x_train_text_tokenized[:3])
print("="*50)

x_val_text_tokenized = tokenize_using_nltk_TreebankWordTokenizer(x_val_text)

print("Before Tokenization : \n", x_val_text[:3])
print("="*50)
print("After Tokenization : \n", x_val_text_tokenized[:3])

fullgram_x_train_text_tokenized = get_all_ngrams(x_train_text_tokenized)
print("full gram tokenization : \n", fullgram_x_train_text_tokenized[:3])
print("="*50)
fullgram_x_val_text_tokenized = get_all_ngrams(x_val_text_tokenized)
print("full gram tokenization : \n", fullgram_x_val_text_tokenized[:3])
print("="*50)

In [ ]:
# Get how many words inside each text after tokenization
num_of_words_in_each_text = [len(text) for text in fullgram_x_train_text_tokenized]
max_len = max(num_of_words_in_each_text)
print("The max length is: ", max_len)
num_of_words_in_each_text[:10]

In [ ]:
# count how many times these value repeated and sort them
new_dicts = get_keys_that_val_gr_than_num(num_of_words_in_each_text, 1000)
keys = list(new_dicts.keys())
values = list(new_dicts.values())
plt.style.use('dark_background')
fig = plt.gcf()
fig.set_size_inches(18.5, 6)
plt.bar(range(len(new_dicts)), values, tick_label=keys)
plt.show()

# Our CBOW Word2Vec Model

In [ ]:
our_word2vec_model = load_word2vec_model("../word2vec_models/rezk_unigram_CBOW_model/train_word2vec_cbow__window_3_min_count_300")

In [ ]:
number_of_features = 300
max_len_str = 128
word2vec_path = "rezk/"
model_path_to_save = "../ml_models_saved/"
estimators = voting_models()

X_train_embed_matrix = text_to_matrix_using_word2vec(our_word2vec_model, fullgram_x_train_text_tokenized, max_len_str)
X_val_embed_matrix = text_to_matrix_using_word2vec(our_word2vec_model, fullgram_x_val_text_tokenized, max_len_str)

print(X_train_embed_matrix.shape)
print("="*50)
print(X_val_embed_matrix.shape)
print("="*50)

In [ ]:
# Train Logistic Regression

model = LogisticRegression(penalty='l2', C=1, multi_class='multinomial', solver='lbfgs', verbose=1)
model = ml_classifer_pipeline(model, X_train_embed_matrix, y_train, X_val_embed_matrix, y_val,word2vec_path, model_path_to_save)

print("=" * 50)

# Train SVC

model = LinearSVC(C=0.5,  max_iter=50, verbose=1)
model = ml_classifer_pipeline(model, X_train_embed_matrix, y_train, X_val_embed_matrix, y_val,word2vec_path, model_path_to_save)

# Hard Voting

print("=" * 50)

model = VotingClassifier(estimators, voting="hard")
model = ml_classifer_pipeline(model, X_train_embed_matrix, y_train, X_val_embed_matrix, y_val,word2vec_path, model_path_to_save)

# AdaBoost 
print("=" * 50)


model = LinearSVC(C=0.5,  verbose=1)
model = AdaBoostClassifier(model,  algorithm="SAMME", n_estimators=5)
model = ml_classifer_pipeline(model, X_train_embed_matrix, y_train, X_val_embed_matrix, y_val,word2vec_path, model_path_to_save)

#  Gradient Boosting
print("=" * 50)

model = GradientBoostingClassifier(n_estimators=10, subsample=.1, learning_rate=.5,   max_depth=5, verbose=1)
model = ml_classifer_pipeline(model, X_train_embed_matrix, y_train, X_val_embed_matrix, y_val,word2vec_path, model_path_to_save)
print("=" * 50)


In [ ]:
bakr_word2vec_model = load_word2vec_model("../word2vec_models/rezk_unigram_CBOW_model/train_word2vec_cbow__window_3_min_count_300")

In [ ]:
word2vec_path = "bakr/"
model_path_to_save = "../ml_models_saved/"
estimators = voting_models()

X_train_embed_matrix = text_to_matrix_using_word2vec(bakr_word2vec_model, fullgram_x_train_text_tokenized, max_len_str)
X_val_embed_matrix = text_to_matrix_using_word2vec(bakr_word2vec_model, fullgram_x_val_text_tokenized, max_len_str)

print(X_train_embed_matrix.shape)
print("="*50)
print(X_val_embed_matrix.shape)
print("="*50)

In [ ]:
# Train Logistic Regression

model = LogisticRegression(penalty='l2', C=1, multi_class='multinomial', solver='lbfgs', verbose=1)
model = ml_classifer_pipeline(model, X_train_embed_matrix, y_train, X_val_embed_matrix, y_val,word2vec_path, model_path_to_save)

print("=" * 50)

# Train SVC

model = LinearSVC(C=0.5,  max_iter=50, verbose=1)
model = ml_classifer_pipeline(model, X_train_embed_matrix, y_train, X_val_embed_matrix, y_val,word2vec_path, model_path_to_save)

# Hard Voting

print("=" * 50)

model = VotingClassifier(estimators, voting="hard")
model = ml_classifer_pipeline(model, X_train_embed_matrix, y_train, X_val_embed_matrix, y_val,word2vec_path, model_path_to_save)

# AdaBoost 

print("=" * 50)

model = LinearSVC(C=0.5,  verbose=1)
model = AdaBoostClassifier(model,  algorithm="SAMME", n_estimators=5)
model = ml_classifer_pipeline(model, X_train_embed_matrix, y_train, X_val_embed_matrix, y_val,word2vec_path, model_path_to_save)
print("=" * 50)

#  Gradient Boosting

model = GradientBoostingClassifier(n_estimators=10, subsample=.1, learning_rate=.5,   max_depth=5, verbose=1)
model = ml_classifer_pipeline(model, X_train_embed_matrix, y_train, X_val_embed_matrix, y_val,word2vec_path, model_path_to_save)
print("=" * 50)


In [ ]:
muhammed_word2vec_model = load_word2vec_model("../word2vec_models/rezk_unigram_CBOW_model/train_word2vec_cbow__window_3_min_count_300")

In [ ]:
word2vec_path = "muhammed/"
model_path_to_save = "../ml_models_saved/"
estimators = voting_models()

X_train_embed_matrix = text_to_matrix_using_word2vec(muhammed_word2vec_model, fullgram_x_train_text_tokenized, max_len_str)
X_val_embed_matrix = text_to_matrix_using_word2vec(muhammed_word2vec_model, fullgram_x_val_text_tokenized, max_len_str)

print(X_train_embed_matrix.shape)
print("="*50)
print(X_val_embed_matrix.shape)
print("="*50)

In [ ]:
# Train Logistic Regression

model = LogisticRegression(penalty='l2', C=1, multi_class='multinomial', solver='lbfgs', verbose=1)
model = ml_classifer_pipeline(model, X_train_embed_matrix, y_train, X_val_embed_matrix, y_val,word2vec_path, model_path_to_save)

print("=" * 50)

# Train SVC

model = LinearSVC(C=0.5,  max_iter=50, verbose=1)
model = ml_classifer_pipeline(model, X_train_embed_matrix, y_train, X_val_embed_matrix, y_val,word2vec_path, model_path_to_save)

# Hard Voting

print("=" * 50)

model = VotingClassifier(estimators, voting="hard")
model = ml_classifer_pipeline(model, X_train_embed_matrix, y_train, X_val_embed_matrix, y_val,word2vec_path, model_path_to_save)

# AdaBoost 

print("=" * 50)

model = LinearSVC(C=0.5,  verbose=1)
model = AdaBoostClassifier(model,  algorithm="SAMME", n_estimators=5)
model = ml_classifer_pipeline(model, X_train_embed_matrix, y_train, X_val_embed_matrix, y_val,word2vec_path, model_path_to_save)
print("=" * 50)

#  Gradient Boosting

model = GradientBoostingClassifier(n_estimators=10, subsample=.1, learning_rate=.5,   max_depth=5, verbose=1)
model = ml_classifer_pipeline(model, X_train_embed_matrix, y_train, X_val_embed_matrix, y_val,word2vec_path, model_path_to_save)
print("=" * 50)


# Load best model & predict test set

In [ ]:
strat_test_set = read_file("test/strat_test_set.csv")
strat_test_set.head()

In [ ]:
X_test_text = list(strat_test_set['text'])
y_test = strat_test_set['label'].values

In [ ]:
X_test_text_tokenized = tokenize_using_nltk_TreebankWordTokenizer(X_test_text)

print("Before Tokenization : \n", X_test_text[:3])
print("="*50)
print("After Tokenization : \n", X_test_text_tokenized[:3])
print("="*50)

fullgram_X_test_text_tokenized = get_all_ngrams(X_test_text_tokenized)
print("full gram tokenization : \n", fullgram_X_test_text_tokenized[:3])
print("="*50)

X_test_embed_matrix = text_to_matrix_using_word2vec(our_word2vec_model, fullgram_X_test_text_tokenized, max_len_str)

print(X_test_embed_matrix.shape)
print("="*50)

In [ ]:
rezk_model = pickle_load_model("ml_models_saved/bakr/" + )
bakr_model = pickle_load_model("ml_models_saved/rezk/" + )
muhammed_model = pickle_load_model("ml_models_saved/muhammed/" + )

X_test_embed_matrix = text_to_matrix_using_word2vec(our_word2vec_model, fullgram_X_test_text_tokenized, max_len_str)
f1_score_result(rezk_model, X_test_embed_matrix, y_test)
print("="*50)

X_test_embed_matrix = text_to_matrix_using_word2vec(bakr_word2vec_model, fullgram_X_test_text_tokenized, max_len_str)
f1_score_result(bakr_model, X_test_embed_matrix, y_test)
print("="*50)

X_test_embed_matrix = text_to_matrix_using_word2vec(muhammed_word2vec_model, fullgram_X_test_text_tokenized, max_len_str)
f1_score_result(muhammed_model, X_test_embed_matrix, y_test)

# Qualitative Evaluation

In [ ]:
print("The number of words the model learn from your dataset are:", len(list(our_word2vec_model.wv.index_to_key)))
print("The number of words the model learn from your dataset are:", len(list(bakr_word2vec_model.wv.index_to_key)))
print("The number of words the model learn from your dataset are:", len(list(muhammed_word2vec_model.wv.index_to_key)))

In [ ]:
our_word2vec_model.wv.most_similar('قابس')

In [ ]:
bakr_word2vec_model.wv.most_similar('قابس')

In [ ]:
muhammed_word2vec_model.wv.most_similar('قابس')

In [ ]:
our_word2vec_model.wv.most_similar('طنطا')

In [ ]:
bakr_word2vec_model.wv.most_similar('طنطا')

In [ ]:
muhammed_word2vec_model.wv.most_similar('طنطا')

In [ ]:
our_word2vec_model.wv.most_similar('فندق')

In [ ]:
bakr_word2vec_model.wv.most_similar('فندق')

In [ ]:
muhammed_word2vec_model.wv.most_similar('فندق')

In [ ]:
our_word2vec_model.wv.most_similar('🤣')

In [ ]:
bakr_word2vec_model.wv.most_similar('🤣')

In [ ]:
muhammed_word2vec_model.wv.most_similar('🤣')

In [ ]:
our_word2vec_model.wv.most_similar('🇰🇼')

In [ ]:
bakr_word2vec_model.wv.most_similar('🇰🇼')

In [ ]:
muhammed_word2vec_model.wv.most_similar('🇰🇼')

In [ ]:
our_word2vec_model.wv.most_similar('IT')

In [ ]:
bakr_word2vec_model.wv.most_similar('IT')

In [ ]:
muhammed_word2vec_model.wv.most_similar('IT')

In [ ]:
our_word2vec_model.wv.most_similar('؟')

In [ ]:
bakr_word2vec_model.wv.most_similar('؟')

In [ ]:
muhammed_word2vec_model.wv.most_similar('؟')

In [ ]:
our_word2vec_model.wv.most_similar('🌸')

In [ ]:
bakr_word2vec_model.wv.most_similar('🌸')

In [ ]:
muhammed_word2vec_model.wv.most_similar('🌸')

In [ ]:
our_word2vec_model.wv.most_similar('كورونا')

In [ ]:
bakr_word2vec_model.wv.most_similar('كورونا')

In [ ]:
muhammed_word2vec_model.wv.most_similar('كورونا')

In [ ]:
our_word2vec_model.wv.most_similar('كرونا')

In [ ]:
bakr_word2vec_model.wv.most_similar('كرونا')

In [ ]:
muhammed_word2vec_model.wv.most_similar('كرونا')

In [ ]:
our_word2vec_model.wv.most_similar('covid')

In [ ]:
bakr_word2vec_model.wv.most_similar('covid')

In [ ]:
muhammed_word2vec_model.wv.most_similar('covid')

In [ ]:
our_word2vec_model.wv.most_similar('covid_19')

In [ ]:
bakr_word2vec_model.wv.most_similar('covid_19')

In [ ]:
muhammed_word2vec_model.wv.most_similar('covid_19')

In [ ]:
our_word2vec_model.wv.most_similar('بايدن')

In [ ]:
bakr_word2vec_model.wv.most_similar('بايدن')

In [ ]:
muhammed_word2vec_model.wv.most_similar('بايدن')

In [ ]:
our_word2vec_model.wv.most_similar('عمر_بن_الخطاب')

In [ ]:
bakr_word2vec_model.wv.most_similar('عمر_بن_الخطاب')

In [ ]:
muhammed_word2vec_model.wv.most_similar('عمر_بن_الخطاب')

In [ ]:
our_word2vec_model.wv.most_similar('ابو_تريكه')

In [ ]:
word2vec_model.wv.most_similar('ابو_تريكه')

In [ ]:
muhammed_word2vec_model.wv.most_similar('ابو_تريكه')

In [ ]:
our_word2vec_model.wv.most_similar('بن_خلدون')

In [ ]:
bakr_word2vec_model.wv.most_similar('بن_خلدون')

In [ ]:
muhammed_word2vec_model.wv.most_similar('بن_خلدون')

In [ ]:
# Reduce the dimension of SENTIMENT_WORDS
tsne_df_scale = tsne_graph(our_word2vec_model, SENTIMENT_WORDS, 500, .03)

_ = init_graph_style()

_ = word_display(tsne_df_scale, SENTIMENT_WORDS, "SENTIMENT_WORDS.png")

In [ ]:
# Reduce the dimension of SENTIMENT_WORDS
tsne_df_scale = tsne_graph(muhammed_word2vec_model, SENTIMENT_WORDS, 500, .03)

_ = init_graph_style()

_ = word_display(tsne_df_scale, SENTIMENT_WORDS, "SENTIMENT_WORDS.png")

In [ ]:
# Reduce the dimension of SENTIMENT_WORDS
tsne_df_scale = tsne_graph(bakr_word2vec_model, SENTIMENT_WORDS, 500, .03)

_ = init_graph_style()

_ = word_display(tsne_df_scale, SENTIMENT_WORDS, "SENTIMENT_WORDS.png")

In [ ]:
# Reduce the dimension of NER_WORDS
tsne_df_scale = tsne_graph(our_word2vec_model, NER_WORDS, 1400, .06)

_ = init_graph_style(figsize=(16, 10))

_ = word_display(tsne_df_scale, NER_WORDS, "NER_WORDS.png")

In [ ]:
# Reduce the dimension of NER_WORDS
tsne_df_scale = tsne_graph(bakr_word2vec_model, NER_WORDS, 1400, .06)

_ = init_graph_style(figsize=(16, 10))

_ = word_display(tsne_df_scale, NER_WORDS, "NER_WORDS.png")

In [ ]:
# Reduce the dimension of NER_WORDS
tsne_df_scale = tsne_graph(muhammed_word2vec_model, NER_WORDS, 1400, .06)

_ = init_graph_style(figsize=(16, 10))

_ = word_display(tsne_df_scale, NER_WORDS, "NER_WORDS.png")